# Automate Event Planning

```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Crew, Task

In [ ]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [ ]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [ ]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [ ]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [ ]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [ ]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [ ]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

2024-08-19 18:02:02,061 - 140133236640640 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 2**:
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: Top San Francisco Venues ; The Westin St. Francis. 335 Powell St, San Francisco, CA 94102, USA. Spaces for 60-1,500 Guests. 8 Hotels. 8 Ballrooms ; Pier 27. The ...
---
Title: Party Venues near San Francisco, CA - Eventective
Link: https://www.eventective.com/san-francisco-ca/party-event-venues/
Snippet: Find and contact local Party Venues in San Francisco, CA with pricing a

Thought: 
I have gathered information about several potential venues for the Tech Innovation Conference in San Francisco. I need to review the details of each venue to choose the most suitable one.

Action: Read website content
Action Input: {"website_url": "https://www.bizzabo.com/blog/san-francisco-event-venues"} 

The 25 Best San Francisco Event Spaces | Bizzabo
Ready to play? Watch our on-demand webinar, Level Up: Unlocking the Power of Gamification to Boost Event ROI. WATCH NOW
ProductsEvent Experience OS
Menu
Event Software Overview
Klik Experiential & SmartBadge
Product Integrations
Security & Compliance
Enterprise Event Software
Insights & Reporting
Capabilities
Event Lead Capture App
Room Block Management
Event Content
Event Networking
Event Marketing
Audience Engagement
Onsite Software & Wearables
Event Live Streaming
Sponsors & Exhibitors
Mobile Event App
Event Registration Software
close
Event Software Overview
Klik Experiential & SmartBadge
Security & Compliance
Enterprise

This is the agent final answer: Based on my research, I have identified several potential venues for the Tech Innovation Conference in San Francisco. The following venues offer a range of amenities and capacities that can accommodate the event:

1. City View at METREON
Address: SoMa
Capacity: 2,000
Website: www.cityviewmetreon.com

2. The Village Venue
Address: Mid-Market
Capacity: 1,100
Website: www.969market.com

3. Exploratorium
Address: Pier 15
Capacity: 3,500
Website: www.exploratorium.edu/rentals

4. The Fillmore
Address: Fillmore District
Capacity: 800
Website: www.livenation.com/venue

5. Gallery 308 at Fort Mason
Address: Marina District
Capacity: 430
Website: https://fortmason.org/venue/gallery-308

6. Bespoke
Address: Downtown
Capacity: 1,200
Website: www.bespokesf.co/events

7. Merchants Exchange’s Julia Morgan Ballroom
Address: Financial District
Capacity: 1,000
Website: https://juliamorganballroom.com

8. The Bently Reserve
Address: Downtown
Capacity: 805
Website: https:/

 


Search results: Title: 2024 Cybersecurity & Technology Innovation Conference
Link: https://www.energy.gov/cio/2024-cybersecurity-technology-innovation-conference
Snippet: Date/Location: May 13-16, 2019, in Denver, CO. Theme: “Powering Cybersecurity and Information Technology Innovation”. 2018. Date/Location: June 4-7, 2018 in ...
---
Title: Technology & Innovation Summit North America - Forrester
Link: https://www.forrester.com/event/technology-innovation-north-america/
Snippet: Join your fellow technology and data leaders at this year's Technology & Innovation Summit in Austin, TX, Sept 9-12, 2024.
---
Title: Tech Conferences: The Best Tech Events Guide for 2024 - Bizzabo
Link: https://www.bizzabo.com/blog/technology-events
Snippet: This carefully curated conference directory features more than 100 tech conferences and events happening globally in 2024. Check it out!
---
Title: 2024 Virtual NWA Technology, Innovation, and Vendor ...
Link: https://www.nwica.org/events/info/2024-vir

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

 


Search results: Title: TechConnect World 2024
Link: https://techconnectworld.com/
Snippet: TechConnect World 2024 will take place June 17-19, 2024, Washington DC, Gaylord National Harbor.
---
Title: Technology & Innovation Summit North America - Forrester
Link: https://www.forrester.com/event/technology-innovation-north-america/
Snippet: Join your fellow technology and data leaders at this year's Technology & Innovation Summit in Austin, TX, Sept 9-12, 2024.
---
Title: Tech Conferences: The Best Tech Events Guide for 2024 - Bizzabo
Link: https://www.bizzabo.com/blog/technology-events
Snippet: This carefully curated conference directory features more than 100 tech conferences and events happening globally in 2024. Check it out!
---
Title: Insurance Tech & Innovation Conference | June 11 - 12, 2024 - USA
Link: https://events.altaworld.tech/Insurance-Tech-Innovation-Conference.html
Snippet: Event Schedule. Here is our event schedule. Day 1 - June 12 2024 ... Stay updated with latest s

Thought:
I need to find more information about the Tech Innovation Conference to effectively promote it to potential attendees.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference 2024 schedule and speakers"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Final Answer:

# Tech Innovation Conference 2024 Promotion Report

## Marketing Activities:
- Researched various tech conferences happening in 2024 to understand the landscape.
- Identified potential competitors and their event details to strategize our promotion.
- Explored the possibility of virtual conferences and their impact on attendee engagement.
- Looked into the best tech events guide for 2024 to gather insights on successful event strategies.
- Analyzed the top innovation conferences of 2024 to benchmark our conference against industry standards.

## Attendee Engagement:
- Investigated the schedule and speakers of similar tech con

- Display the generated `venue_details.json` file.

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'SoMa, San Francisco',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement Report

### Tech Innovation Conference Details:
- **Name:** Tech Innovation Conference
- **Date:** To be announced
- **Location:** The Pearl, 601 19th St, San Francisco, CA 94107, USA
- **Capacity:** 500
- **Booking Status:** Available

### Marketing Strategy:
- **Website:** Visit [Tech Innovation Conference Website](https://www.techinnovationconference.com) for more details.
- **Search Engine Optimization (SEO):** Implement SEO strategies to increase visibility in search results.
- **Social Media Campaigns:** Create engaging posts on various platforms to attract potential attendees.
- **Email Marketing:** Send personalized emails to targeted audiences to encourage registration.
- **Collaborations:** Partner with tech influencers and industry leaders for promotion.
- **Content Marketing:** Develop informative content about the event to generate interest.

### Attendee Engagement:
- **Interactive Sessions:** Plan interactive sessions and workshops to keep attendees engaged.
- **Networking Opportunities:** Provide networking sessions for participants to connect with industry professionals.
- **Q&A Panels:** Host Q&A panels with keynote speakers to facilitate interaction.
- **Virtual Reality Experience:** Offer a virtual reality experience to showcase innovative technology.
- **Feedback Surveys:** Collect feedback to improve future events and enhance attendee experience.

### Key Performance Indicators (KPIs):
- **Registrations:** Track the number of registrations to measure event interest.
- **Social Media Engagement:** Monitor likes, shares, and comments on social media posts.
- **Email Open Rates:** Measure email open rates to gauge interest in event updates.
- **Attendee Feedback:** Evaluate post-event surveys to assess attendee satisfaction.

### Conclusion:
By implementing a comprehensive marketing strategy and focusing on attendee engagement, the Tech Innovation Conference aims to attract over 500 potential attendees and create a memorable experience for all participants.